# LangChain Expression Language (LCEL)

In [65]:
import openai
import os
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [66]:

# The base URL for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.
os.environ['AZURE_OPENAI_ENDPOINT']
# The API key for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.
os.environ['AZURE_OPENAI_API_KEY']
os.environ["OPENAI_API_VERSION"] = "2023-12-01-preview"
model=os.environ["DEPLOYMENT_NAME"]

In [67]:
#!pip install pydantic==1.10.8

In [68]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [69]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
llm = AzureChatOpenAI(deployment_name=model, temperature=0.7)

model = llm
output_parser = StrOutputParser()

In [70]:
chain = prompt | model | output_parser

In [71]:
chain.invoke({"topic": "bears"})

'Why do bears never wear shoes?\n\nBecause they have bear feet!'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [72]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [73]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [74]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [75]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [76]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [77]:
from langchain.schema.runnable import RunnableMap

In [78]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [79]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [80]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [81]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

In [82]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [84]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = AzureChatOpenAI(deployment_name=os.environ["DEPLOYMENT_NAME"],temperature=0).bind(functions=functions)

In [85]:
runnable = prompt | model

In [86]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 64, 'total_tokens': 81}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run-b426e0e0-bdcb-4bc5-a454-8751baeea874-0')

In [87]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [88]:
model = model.bind(functions=functions)

In [89]:
runnable = prompt | model

In [90]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "team_name": "patriots"\n}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 99, 'total_tokens': 118}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run-18ac47e0-2e46-4549-9741-901caabb9ee3-0')

## Fallbacks

In [62]:
from langchain.llms import AzureOpenAI
import json

**Note**: Due to the deprecation of OpenAI's model `text-davinci-001` on 4 January 2024, you'll be using OpenAI's recommended replacement model `gpt-3.5-turbo-instruct` instead.

In [95]:
simple_model = AzureChatOpenAI(
    temperature=0,max_tokens=1000, 
    model=os.environ["DEPLOYMENT_NAME"]
)
simple_chain = simple_model | json.loads

In [96]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [97]:
simple_model.invoke(challenge)

AIMessage(content='{\n  "poem1": {\n    "title": "Whispers of the Wind",\n    "author": "Emily Rivers",\n    "first_line": "Softly it comes, the whisper of the wind"\n  },\n  "poem2": {\n    "title": "Silent Serenade",\n    "author": "Jacob Moore",\n    "first_line": "In the stillness of night, a silent serenade"\n  },\n  "poem3": {\n    "title": "Dancing Shadows",\n    "author": "Sophia Anderson",\n    "first_line": "Shadows dance upon the walls, a secret ballet"\n  }\n}', response_metadata={'token_usage': {'completion_tokens': 136, 'prompt_tokens': 31, 'total_tokens': 167}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'h

**Note**: The next line is expected to fail.

In [98]:
simple_chain.invoke(challenge)

TypeError: the JSON object must be str, bytes or bytearray, not AIMessage

In [101]:
model = AzureChatOpenAI(temperature=0,model=os.environ["DEPLOYMENT_NAME"])
chain = model | StrOutputParser() | json.loads

In [102]:
chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [103]:
final_chain = simple_chain.with_fallbacks([chain])

In [104]:
final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

## Interface

In [105]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = AzureChatOpenAI(model=os.environ["DEPLOYMENT_NAME"])
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [106]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

In [107]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 'Why don\'t frogs make good drivers? They always "jump" the red light!']

In [108]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!



In [109]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears wear shoes?\n\nBecause they have bear feet!"